<a href="https://colab.research.google.com/github/LC1332/Luotuo-Text-Embedding/blob/main/gradio/embed_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
! pip install -U kaleido
! pip install gradio
! pip install transformers
! pip install openTSNE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 31.6 MB/s eta 0:00:00


In [2]:
! git clone https://github.com/LC1332/Luotuo-Text-Embedding

Cloning into 'Luotuo-Text-Embedding'...
remote: Enumerating objects: 474, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 474 (delta 43), reused 47 (delta 14), pack-reused 388
Receiving objects: 100% (474/474), 30.78 MiB | 27.92 MiB/s, done.
Resolving deltas: 100% (270/270), done.


In [4]:
%cd Luotuo-Text-Embedding/gradio

/content/Luotuo-Text-Embedding/gradio


In [5]:
import gradio as gr
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
import torch
# Import our models. The package will take care of downloading the models automatically

In [9]:
from divide import divide_inputs,get_all_embedding
from tsne import TSNE_Plot
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None,
                       pooler_type="cls",
                       temp=0.05,
                       mlp_only_train=False,
                       init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert",
                                  trust_remote_code=True,
                                  model_args=model_args)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [10]:
def generate_image(text_input):
    # 将输入的文本按行分割并保存到列表中
    text_left, text_right = divide_inputs(text_input)
    embeddings_left, embeddings_right = get_all_embedding(
        model, tokenizer, text_left, text_right)
    merged_list = text_left + text_right
    merged_embed = torch.cat((embeddings_left, embeddings_right), dim=0)
    tsne_plot = TSNE_Plot(merged_list, merged_embed, n_clusters=4)
    fig = tsne_plot.tsne_plot(n_sentence=40)
    return fig

In [ ]:
# 创建 Gradio 界面
iface = gr.Interface(fn=generate_image,
                     inputs=gr.inputs.Textbox(lines=50,
                                              placeholder='在此输入文本，一行一句...'),
                     outputs=gr.outputs.Image('pil'),
                     layout="vertical")


iface.launch(share = True,debug=True)

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://12710f799a6055d08a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
